In [4]:
import importlib
import pickle

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import warnings

warnings.filterwarnings("ignore")
sns.set_style()
plt.rcParams["figure.figsize"] = (20, 10)


# Importacion de Modelos Entrenados

Debido a que se entrenaron todos los modelos en el Hito 2, se procedera a importar cada uno de los
modelos ya entrenados.

In [5]:
archivos_a_leer = [
    "modelo_logistic.pickle",
    "modelo_naive_bayes.pickle",
    "modelo_arbol.pickle",
    "modelo_random_forest.pickle",
    "modelo_gradient_boosting.pickle",
]

modelos = {}
for nombre_archivo in archivos_a_leer:
    with open(nombre_archivo, "rb") as file:
        modelo = pickle.load(file)
        modelos[nombre_archivo.split(".")[0]] = modelo

# Reporte comparativo de desempenos

En el Hito 2 se reportaron los desempenos de cada uno de los modelos ya entrenados. Esto se realizo
tanto para el conjunto de entrenamiento y el conjunto de validacion. En la siguiente tabla se
recopila a modo de resumen los resultados.

|Modelo|Entrenamiento Accuracy|Validacion Accuracy|Validacion ROC|
|------|----------------------|-------------------|--------------|
|Logistico|0.67|0.68|0.68|
|Naive Bayes|0.67|0.68|0.68|
|Arbol de clasificacion|0.57|0.57|0.56|
|Random Forest|0.66|0.66|0.65|
|Gradient Boosting|0.66|0.67|0.66|


- Se puede observar que los mejores modelos de clasificacion (segun el valor ROC) fueron el
modelo de regresion logistica o naive bayes. Ambos modelos presentaron un desempeno identico,
tanto en el conjunto de entrenamiento como en el de validacion. Ademas, fueron los modelos que
tuvieron el menor coste computacional, demorando ~8 y ~2 segundos en entrenar, respectivamente.

- El peor modelo fue el arbol de clasificacion, con un valor de ROC de 0.56. Por lo tanto, clasifica
en 6% mejor que un clasificador aleatorio.

- El modelo de Random Forest y Gradient Boosting fueron los modelos mas costosos computacionalmente,
demorando ~3 y ~9 minutos en entrenar, respectivamente. A pesar de su alto costo computacional,
estos modelos tuvieron un resultado comparable (levemente peor) al modelo logistico o naive bayes.

- El valor ROC promedio de los 5 modelos fue de 0.65.

# Exportacion de Mejores Modelos

Teniendo en cuenta los resultados anteriores, los mejores modelos son el modelo de regresion
logistica y naive bayes. Por lo tanto, ambos seran exportados para validacion externa.

In [11]:
modelos_a_exportar = {
    'modelo_1_modelo_logistico.pickle': modelos['modelo_logistic'],
    'modelo_2_modelo_naive_bayes.pickle': modelos['modelo_naive_bayes'],
}

In [13]:
for archivo, modelo in modelos_a_exportar.items():
    with open(archivo, 'wb') as file:
        pickle.dump(modelo, file)